In [17]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
import pandas as pd
data= pd.read_csv('sample.csv')
data.head()

,news
0,Virat scored century in match
1,BJP won in election
2,Bumra took five wickets in a match
3,Congress form state government


In [19]:
def preprocessing_pipeline(text):
    text=text.lower()
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F" 
        "\U0001F300-\U0001F5FF" 
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f" 
        "\u3030"
        "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text
data['preprocessed_text'] = data['news'].apply(preprocessing_pipeline)
data.head()

,news,preprocessed_text
0,Virat scored century in match,virat scored century match
1,BJP won in election,bjp election
2,Bumra took five wickets in a match,bumra took five wicket match
3,Congress form state government,congress form state government


In [20]:
def bag_of_words(corpus):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    try:
        feature_names = vectorizer.get_feature_names_out()
    except AttributeError:
        feature_names = vectorizer.get_feature_names()
    bow_df = pd.DataFrame(X.toarray(), columns=feature_names)
    return bow_df

In [21]:
bag_of_words_df = bag_of_words(data['preprocessed_text'])
print(bag_of_words_df.head(10))

   bjp  bumra  century  congress  election  five  form  government  match  \
0    0      0        1         0         0     0     0           0      1   
1    1      0        0         0         1     0     0           0      0   
2    0      1        0         0         0     1     0           0      1   
3    0      0        0         1         0     0     1           1      0   

   scored  state  took  virat  wicket  
0       1      0     0      1       0  
1       0      0     0      0       0  
2       0      0     1      0       1  
3       0      1     0      0       0  


In [22]:
from sklearn.decomposition import LatentDirichletAllocation
num_topics = 2
LDA=LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(bag_of_words_df)

,"n_components n_components: int, default=10Number of topics... versionchanged:: 0.19 ``n_topics`` was renamed to ``n_components``",2
,"doc_topic_prior doc_topic_prior: float, default=NonePrior of document topic distribution `theta`. If the value is None,defaults to `1 / n_components`.In [1]_, this is called `alpha`.",None
,"topic_word_prior topic_word_prior: float, default=NonePrior of topic word distribution `beta`. If the value is None, defaultsto `1 / n_components`.In [1]_, this is called `eta`.",None
,"learning_method learning_method: {'batch', 'online'}, default='batch'Method used to update `_component`. Only used in :meth:`fit` method.In general, if the data size is large, the online update will be muchfaster than the batch update.Valid options:- 'batch': Batch variational Bayes method. Use all training data in each EM update. Old `components_` will be overwritten in each iteration.- 'online': Online variational Bayes method. In each EM update, use mini-batch of training data to update the ``components_`` variable incrementally. The learning rate is controlled by the ``learning_decay`` and the ``learning_offset`` parameters... versionchanged:: 0.20 The default learning method is now ``""batch""``.",'batch'
,"learning_decay learning_decay: float, default=0.7It is a parameter that control learning rate in the online learningmethod. The value should be set between (0.5, 1.0] to guaranteeasymptotic convergence. When the value is 0.0 and batch_size is``n_samples``, the update method is same as batch learning. In theliterature, this is called kappa.",0.7
,"learning_offset learning_offset: float, default=10.0A (positive) parameter that downweights early iterations in onlinelearning. It should be greater than 1.0. In the literature, this iscalled tau_0.",10.0
,"max_iter max_iter: int, default=10The maximum number of passes over the training data (aka epochs).It only impacts the behavior in the :meth:`fit` method, and not the:meth:`partial_fit` method.",10
,"batch_size batch_size: int, default=128Number of documents to use in each EM iteration. Only used in onlinelearning.",128
,"evaluate_every evaluate_every: int, default=-1How often to evaluate perplexity. Only used in `fit` method.set it to 0 or negative number to not evaluate perplexity intraining at all. Evaluating perplexity can help you check convergencein training process, but it will also increase total training time.Evaluating perplexity in every iteration might increase training timeup to two-fold.",-1
,"total_samples total_samples: int, default=1e6Total number of documents. Only used in the :meth:`partial_fit` method.",1000000.0
,"perp_tol perp_tol: float, default=1e-1Perplexity tolerance. Only used when ``evaluate_every`` is greater than 0.",0.1


In [23]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        top_features_ind = topic.argsort()[:-num_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        print(" ".join(top_features))
    print("\n")


In [24]:
num_top_words = 15
print(f"Top {num_top_words} words for each topic:")
display_topics(LDA, bag_of_words_df.columns, num_top_words)

Top 15 words for each topic:
Topic 0:
match five took wicket bumra virat century scored election bjp state form government congress
Topic 1:
congress government state form bjp election scored century virat match bumra wicket took five




In [25]:
data['news']= data['news'].astype(str)
data['LDA_Topic'] = LDA.transform(bag_of_words_df).argmax(axis=1)
print(data[['news', 'LDA_Topic']].head())

                                 news  LDA_Topic
0       Virat scored century in match          0
1                 BJP won in election          0
2  Bumra took five wickets in a match          0
3      Congress form state government          1


In [26]:
from sklearn.decomposition import NMF
num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(bag_of_words_df)

,"n_components n_components: int or {'auto'} or None, default='auto'Number of components. If `None`, all features are kept.If `n_components='auto'`, the number of components is automatically inferredfrom W or H shapes... versionchanged:: 1.4 Added `'auto'` value... versionchanged:: 1.6 Default value changed from `None` to `'auto'`.",2
,"init init: {'random', 'nndsvd', 'nndsvda', 'nndsvdar', 'custom'}, default=NoneMethod used to initialize the procedure.Valid options:- `None`: 'nndsvda' if n_components <= min(n_samples, n_features), otherwise random.- `'random'`: non-negative random matrices, scaled with: `sqrt(X.mean() / n_components)`- `'nndsvd'`: Nonnegative Double Singular Value Decomposition (NNDSVD) initialization (better for sparseness)- `'nndsvda'`: NNDSVD with zeros filled with the average of X (better when sparsity is not desired)- `'nndsvdar'` NNDSVD with zeros filled with small random values (generally faster, less accurate alternative to NNDSVDa for when sparsity is not desired)- `'custom'`: Use custom matrices `W` and `H` which must both be provided... versionchanged:: 1.1 When `init=None` and n_components is less than n_samples and n_features defaults to `nndsvda` instead of `nndsvd`.",None
,"solver solver: {'cd', 'mu'}, default='cd'Numerical solver to use:- 'cd' is a Coordinate Descent solver.- 'mu' is a Multiplicative Update solver... versionadded:: 0.17 Coordinate Descent solver... versionadded:: 0.19 Multiplicative Update solver.",'cd'
,"beta_loss beta_loss: float or {'frobenius', 'kullback-leibler', 'itakura-saito'}, default='frobenius'Beta divergence to be minimized, measuring the distance between Xand the dot product WH. Note that values different from 'frobenius'(or 2) and 'kullback-leibler' (or 1) lead to significantly slowerfits. Note that for beta_loss <= 0 (or 'itakura-saito'), the inputmatrix X cannot contain zeros. Used only in 'mu' solver... versionadded:: 0.19",'frobenius'
,"tol tol: float, default=1e-4Tolerance of the stopping condition.",0.0001
,"max_iter max_iter: int, default=200Maximum number of iterations before timing out.",200
,"random_state random_state: int, RandomState instance or None, default=NoneUsed for initialisation (when ``init`` == 'nndsvdar' or'random'), and in Coordinate Descent. Pass an int for reproducibleresults across multiple function calls.See :term:`Glossary `.",42
,"alpha_W alpha_W: float, default=0.0Constant that multiplies the regularization terms of `W`. Set it to zero(default) to have no regularization on `W`... versionadded:: 1.0",0.0
,"alpha_H alpha_H: float or ""same"", default=""same""Constant that multiplies the regularization terms of `H`. Set it to zero tohave no regularization on `H`. If ""same"" (default), it takes the same value as`alpha_W`... versionadded:: 1.0",'same'
,"l1_ratio l1_ratio: float, default=0.0The regularization mixing parameter, with 0 <= l1_ratio <= 1.For l1_ratio = 0 the penalty is an elementwise L2 penalty(aka Frobenius Norm).For l1_ratio = 1 it is an elementwise L1 penalty.For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2... versionadded:: 0.17 Regularization parameter *l1_ratio* used in the Coordinate Descent solver.",0.0
,"verbose verbose: int, default=0Whether to be verbose.",0


In [27]:
print(f"Top {num_top_words} words for each topic using NMF:")
display_topics(nmf_model, bag_of_words_df.columns, num_top_words)

Top 15 words for each topic using NMF:
Topic 0:
match wicket took five bumra virat century scored state government election form congress bjp
Topic 1:
state government form congress election bjp virat century scored match took wicket five bumra




In [28]:
data['news']= data['news'].astype(str)
document_topics = nmf_model.transform(bag_of_words_df)
data['NMF_Topic'] = document_topics.argmax(axis=1)
print(data[['news', 'NMF_Topic']].head())

                                 news  NMF_Topic
0       Virat scored century in match          0
1                 BJP won in election          1
2  Bumra took five wickets in a match          0
3      Congress form state government          1


In [29]:
data.head()

,news,preprocessed_text,LDA_Topic,NMF_Topic
0,Virat scored century in match,virat scored century match,0,0
1,BJP won in election,bjp election,0,1
2,Bumra took five wickets in a match,bumra took five wicket match,0,0
3,Congress form state government,congress form state government,1,1


In [30]:
df=pd.read_csv('arxiv_data.csv')
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [31]:
df['preprocessed_summary'] = df['summaries'].apply(preprocessing_pipeline)
df.head()

,titles,summaries,terms,preprocessed_summary
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']",stereo matching one widely used technique infe...
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']",recent advancement artificial intelligence ai ...
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']",paper proposed novel mutual consistency networ...
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV'],consistency training proven advanced semisuper...
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']",ensure safety automated driving correct percep...


In [32]:
def bag_of_word(corpus):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    try:
        feature_names = vectorizer.get_feature_names_out()
    except AttributeError:
        feature_names = vectorizer.get_feature_names()
    bow_df = pd.DataFrame(X.toarray(), columns=feature_names)
    return bow_df


sample_size = 1000 
bag_of_words_df2 = bag_of_word(df['preprocessed_summary'].head(sample_size))
print(bag_of_words_df2.head(10))

   001074  002  003  00486  005  007fps  0088  01  011  012  ...  youtube  \
0       0    0    0      0    0       0     0   0    0    0  ...        0   
1       0    0    0      0    0       0     0   0    0    0  ...        0   
2       0    0    0      0    0       0     0   0    0    0  ...        0   
3       0    0    0      0    0       0     0   0    0    0  ...        0   
4       0    0    0      0    0       0     0   0    0    0  ...        0   
5       0    0    0      0    0       0     0   0    0    0  ...        0   
6       0    0    0      0    0       0     0   0    0    0  ...        0   
7       0    0    0      0    0       0     0   0    0    0  ...        0   
8       0    0    0      0    0       0     0   0    0    0  ...        0   
9       0    0    0      0    0       0     0   0    0    0  ...        0   

   youtubevos  zebrafish  zernike  zero  zerodice  zijdenbos  zone  zsi  \
0           0          0        0     0         0          0     0    0   
1 

In [49]:
from sklearn.decomposition import LatentDirichletAllocation
num_topics = 5
LDA=LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(bag_of_words_df2)
print(f"Top {num_top_words} words for each topic using LDA:")
display_topics(LDA, bag_of_words_df2.columns, num_top_words)
df.loc[:len(bag_of_words_df2)-1, 'LDA_Topic'] = LDA.transform(bag_of_words_df2).argmax(axis=1)
print(df[['preprocessed_summary', 'LDA_Topic']].head())

Top 15 words for each topic using LDA:
Topic 0:
image segmentation network model method semantic feature approach deep learning based task result neural using
Topic 1:
segmentation image network method proposed feature performance information approach result medical algorithm brain 3d model
Topic 2:
segmentation image learning data model method network training medical task domain performance deep approach propose
Topic 3:
image segmentation object feature proposed algorithm based using method model graph support context query result
Topic 4:
segmentation image method model result network proposed deep based approach learning medical task used performance


                                preprocessed_summary  LDA_Topic
0  stereo matching one widely used technique infe...        0.0
1  recent advancement artificial intelligence ai ...        1.0
2  paper proposed novel mutual consistency networ...        2.0
3  consistency training proven advanced semisuper...        2.0
4  ensure safe

In [51]:
from sklearn.decomposition import NMF
num_topics = 5
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(bag_of_words_df2)
print(f"Top {num_top_words} words for each topic using NMF:")
display_topics(nmf_model, bag_of_words_df2.columns, num_top_words)
df.loc[:len(bag_of_words_df2)-1, 'NMF_Topic'] = nmf_model.transform(bag_of_words_df2).argmax(axis=1)
print(df[['preprocessed_summary', 'NMF_Topic']].head())

Top 15 words for each topic using NMF:
Topic 0:
segmentation method result proposed object algorithm information propose approach semantic based stateoftheart performance label boundary
Topic 1:
image based using method approach feature algorithm used region proposed result segment object pixel paper
Topic 2:
data learning domain training deep medical method annotation image label performance task labeled target unlabeled
Topic 3:
network segmentation neural architecture deep convolutional task feature performance propose image 3d medical proposed layer
Topic 4:
model segmentation task prediction performance different trained semantic learning proposed dataset training datasets output show


                                preprocessed_summary  NMF_Topic
0  stereo matching one widely used technique infe...        3.0
1  recent advancement artificial intelligence ai ...        2.0
2  paper proposed novel mutual consistency networ...        4.0
3  consistency training proven advanced sem